## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time
import googlemaps

## Read-in

In [2]:
df = pd.read_csv('Tauber - FOR MAP.csv')

In [3]:
df = df.rename(columns={'Unnamed: 12':'Deed Link'})

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [5]:
df['Full_Address'] = df['Address'] + ' Chicago, IL'

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## HTML Popup Formatter

In [8]:
pd.set_option('display.max_columns',None)

In [9]:
df.head(1)

,Owner,Address,Number of Units,Purchase Amount,Purchase Date,Mortgage Amount,Loan Originator,Current Mortgage Holder,Still Own?,In Foreclosure?,Other Lawsuits?,Board of Review 2024 value,Deed Link,Full_Address,geocoded,lat,lon
0,ETC5 LLC,9130 S. Throop St.,4,"$680,000",May 2024,"$544,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$340,000",https://crs.cookcountyclerkil.gov/Document/Det...,"9130 S. Throop St. Chicago, IL","(41.7276646, -87.65588939999999)",41.727665,-87.655889


In [10]:
def popup_html(row):
    owner             = row['Owner']
    address           = row['Address']
    num_units         = row['Number of Units']
    purchase_amount   = row['Purchase Amount']
    purchase_date     = row['Purchase Date']
    mortgage_amount   = row['Mortgage Amount']
    loan_originator   = row['Loan Originator']
    current_m_holder  = row['Current Mortgage Holder']
    bor_2024_value    = row['Board of Review 2024 value']
    
    # Format currency fields with commas and two decimals
    fmt = lambda x: f"{x:,.2f}" if isinstance(x, (int, float)) else x

    html = f"""<!DOCTYPE html>
<html>
  <body>
    <strong>Owner:</strong> {owner}<br>
    <strong>Address:</strong> {address}<br>
    <strong>Number of Units:</strong> {num_units}<br>
    <strong>Purchase Amount:</strong> ${fmt(purchase_amount)}<br>
    <strong>Purchase Date:</strong> {purchase_date}<br>
    <strong>Mortgage Amount:</strong> ${fmt(mortgage_amount)}<br>
    <strong>Loan Originator:</strong> {loan_originator}<br>
    <strong>Current Mortgage Holder:</strong> {current_m_holder}<br>
    <strong>Board of Review 2024 Value:</strong> ${fmt(bor_2024_value)}<br>
  </body>
</html>
"""
    return html


In [11]:
%store -r map_box_api_key

In [12]:
df

,Owner,Address,Number of Units,Purchase Amount,Purchase Date,Mortgage Amount,Loan Originator,Current Mortgage Holder,Still Own?,In Foreclosure?,Other Lawsuits?,Board of Review 2024 value,Deed Link,Full_Address,geocoded,lat,lon
0,ETC5 LLC,9130 S. Throop St.,4,"$680,000",May 2024,"$544,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$340,000",https://crs.cookcountyclerkil.gov/Document/Det...,"9130 S. Throop St. Chicago, IL","(41.7276646, -87.65588939999999)",41.727665,-87.655889
1,ETS2 LLC,8017 S. Hermitage Ave.,2,"$495,000",March 2024,"$400,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$180,000",https://crs.cookcountyclerkil.gov/Document/Det...,"8017 S. Hermitage Ave. Chicago, IL","(41.7479542, -87.6667587)",41.747954,-87.666759
2,"CCP1, LLC",5226 S. Paulina St.,4,"$485,000",Dec. 2023,"$364,000",Ark Mortgage,"Wilmington Savings Fund Society, as owner trustee",Y,Y,NaN,"$460,000",https://crs.cookcountyclerkil.gov/Document/Det...,"5226 S. Paulina St. Chicago, IL","(41.7986639, -87.66739489999999)",41.798664,-87.667395
3,ETS2 LLC,8052 S. Langley Ave.,2,"$495,000",Feb. 2024,"$396,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$167,000",https://crs.cookcountyclerkil.gov/Document/Det...,"8052 S. Langley Ave. Chicago, IL","(41.74788360000001, -87.60788389999999)",41.747884,-87.607884
4,ETS LLC,8047 S. Vernon Ave.,3,"$495,000",Feb. 2024,"$496,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$173,000",https://crs.cookcountyclerkil.gov/Document/Det...,"8047 S. Vernon Ave. Chicago, IL","(41.7479549, -87.61334660000001)",41.747955,-87.613347
5,ETC1 LLC,5621 S. Green St.,8,"$680,000",May 2024,"$544,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$235,000",https://crs.cookcountyclerkil.gov/Document/Det...,"5621 S. Green St. Chicago, IL","(41.7918269, -87.6460505)",41.791827,-87.646051
6,ETS1 LLC,7944 S. Eberhart Ave.,2,"$495,000",Feb. 2024,"$371,000",Ark Mortgage,"Wilmington Savings Fund Society, as owner trustee",Y,Y,NaN,"$165,000",https://crs.cookcountyclerkil.gov/Document/Det...,"7944 S. Eberhart Ave. Chicago, IL","(41.7498099, -87.6127876)",41.749810,-87.612788
7,ETS2 LLC,7938 S. Langley Ave.,2,"$495,000",Feb. 2024,"$396,000",Ark Mortgage,Silver Hill Capital,Y,Y,NaN,"$154,000",https://crs.cookcountyclerkil.gov/Document/Det...,"7938 S. Langley Ave. Chicago, IL","(41.7500783, -87.6079344)",41.750078,-87.607934
8,ETS1 LLC,7937 S. Eberhart Ave.,2,"$495,000",Feb. 2024,"$371,000",Ark Mortgage,"Wilmington Savings Fund Society, as owner trustee",Y,Y,NaN,"$148,000",https://crs.cookcountyclerkil.gov/Document/Det...,"7937 S. Eberhart Ave. Chicago, IL","(41.7500776, -87.6121723)",41.750078,-87.612172


In [13]:
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement

# ——— Base map ———
m = folium.Map(
    location=df[['lat','lon']].mean().to_list(),
    zoom_start=12,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Create a MarkerCluster and add it to the map ———
marker_cluster = MarkerCluster(name="Properties").add_to(m)

# ——— Add your markers into that cluster group ———
for _, row in df.iterrows():
    lat, lon = row['lat'], row['lon']
    popup = folium.Popup(popup_html(row), max_width=400)
    # You can still use CircleMarker inside a cluster:
    folium.CircleMarker(
        location=(lat, lon),
        radius=10,
        fill=True,
        fill_color='blue',
        fill_opacity=1,
        color=None,
        popup=popup
    ).add_to(marker_cluster)

# ——— (Optional) Add LayerControl so you can toggle clusters on/off ———
folium.LayerControl().add_to(m)

m  # display


In [14]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [15]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Tauber_08052025
